In [ ]:
# reference : https://github.com/IyatomiLab/LeafGAN

In [1]:
!nvidia-smi

Sun May 29 11:20:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/IyatomiLab/LeafGAN

Cloning into 'LeafGAN'...
remote: Enumerating objects: 239, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 239 (delta 51), reused 50 (delta 50), pack-reused 176
Receiving objects: 100% (239/239), 5.21 MiB | 37.29 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [3]:
%cd /content/LeafGAN

/content/LeafGAN


In [4]:
!pip install -r requirements.txt
!pip install dominate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 676 kB 7.6 MB/s 
     |████████████████████████████████| 47.8 MB 2.3 MB/s 
     |████████████████████████████████| 38.1 MB 235 kB/s 
ERROR: Could not find a version that satisfies the requirement skimage>=0.16.1 (from versions: 0.0)
ERROR: No matching distribution found for skimage>=0.16.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dominate-2.6.0-py2.py3-none-any.whl (29 kB)


In [5]:
!unzip -qq "/content/drive/MyDrive/4-1_Capstone/CapstoneDataset/images_concatenate.zip" -d /content/images

In [6]:
from glob import glob
import os
from natsort import natsorted
images = natsorted(glob("/content/images/*"))

<pre>
'''
 0 : '팥_흰가루병',
 1 : '팥_세균잎마름병',
 2 : '팥_Rhizopus',
 3 : '참깨_세균성점무늬병',
 4 : '참깨_흰가루병',
 5 : '콩_노균병',
 6 : '콩_들불병',
 7 : '콩_불마름병',
 8 : '콩_병징',
 9 : '팥 정상',
10 : '참깨 정상',
11 : '콩 정상',
'''

In [7]:
healthy_label = '10' # 참깨
disease_label = '4' # 참깨_흰가루병

# healthy_label = '11' # 콩
# disease_label = '7' # 콩_불마름병

# healthy_label = '11' # 콩
# disease_label = '6' # 콩_들불병

In [8]:
healthy_images = list(filter(lambda f : f.split('/')[-1].split('_')[0] == healthy_label, images))
disease_images = list(filter(lambda f : f.split('/')[-1].split('_')[0] == disease_label, images))

In [9]:
import numpy as np
import os

os.mkdir("/content/dataset")
os.mkdir("/content/dataset/trainA")
os.mkdir("/content/dataset/testA")
os.mkdir("/content/dataset/trainB")
os.mkdir("/content/dataset/testB")


healthy_len = len(healthy_images)
disease_len = len(disease_images)

healthy_images = np.array(healthy_images)
disease_images = np.array(disease_images)

from sklearn.model_selection import train_test_split
train_A_i, test_A_i, _, _ = train_test_split(list(range(healthy_len)), list(range(healthy_len)), random_state=0)
train_B_i, test_B_i, _, _ = train_test_split(list(range(disease_len)), list(range(disease_len)), random_state=0)

import shutil
for image in healthy_images[train_A_i]:
    file_name = image.split('/')[-1]
    dst = os.path.join("/content/dataset/trainA", file_name)

    shutil.move(image, dst)

for image in healthy_images[test_A_i]:
    file_name = image.split('/')[-1]
    dst = os.path.join("/content/dataset/testA", file_name)

    shutil.move(image, dst)

for image in disease_images[train_B_i]:
    file_name = image.split('/')[-1]
    dst = os.path.join("/content/dataset/trainB", file_name)

    shutil.move(image, dst)

for image in disease_images[test_B_i]:
    file_name = image.split('/')[-1]
    dst = os.path.join("/content/dataset/testB", file_name)

    shutil.move(image, dst)


In [10]:
!python prepare_mask.py --source /content/dataset --pretrain_path "/content/drive/MyDrive/4-1_Capstone/LFLSeg_resnet101.pth" --image_size 224 --threshold 0.35

##### testB #####
  0% 0/38 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1033: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100% 38/38 [00:05<00:00,  6.62it/s]
##### testA #####
100% 142/142 [00:19<00:00,  7.31it/s]
##### trainB #####
100% 113/113 [00:15<00:00,  7.20it/s]
##### trainA #####
100% 425/425 [00:59<00:00,  7.10it/s]
done


In [18]:
from PIL import Image
import numpy as np
img = np.array(Image.open("/content/dataset/testA_mask/10_102.jpg"))

from shutil import make_archive
make_archive("/content/testA_mask", 'zip', "/content/dataset/testA_mask")
make_archive("/content/testB_mask", 'zip', "/content/dataset/testB_mask")
make_archive("/content/trainA_mask", 'zip', "/content/dataset/trainA_mask")
make_archive("/content/trainB_mask", 'zip', "/content/dataset/trainB_mask")

'/content/trainB_mask.zip'

In [ ]:
from glob import glob
import numpy as np
imgs1      = glob("/content/dataset/testA/*")
imgs1_mask = glob("/content/dataset/testA_mask/*")

cnt = 0
from PIL import Image
import matplotlib.pyplot as plt
for img1, img1_mask in zip(imgs1, imgs1_mask):    
    plt.subplot(1, 2, 1)
    img1 = Image.open(img1)
    plt.imshow(img1)
    
    plt.subplot(1, 2, 2)
    img1_mask = Image.open(img1_mask)
    plt.imshow(img1_mask)
    plt.show()
    # if np.array(img1_mask).sum() == 0: cnt+=1


In [ ]:
from glob import glob
import numpy as np
imgs1      = glob("/content/dataset/testA/*")
imgs1_mask = glob("/content/dataset/testA_mask/*")

cnt = 0
from PIL import Image
import matplotlib.pyplot as plt
for img1, img1_mask in zip(imgs1, imgs1_mask):    
    plt.subplot(1, 2, 1)
    img1 = Image.open(img1)
    plt.imshow(img1)
    
    plt.subplot(1, 2, 2)
    img1_mask = Image.open(img1_mask)
    plt.imshow(img1_mask)
    plt.show()
    # if np.array(img1_mask).sum() == 0: cnt+=1


In [ ]:
cnt

31

In [ ]:
cnt

31

In [ ]:
# git clone한 leaf_gan_model은 indentation 오류가 존재함
!cp /content/drive/MyDrive/4-1_Capstone/temp/leaf_gan_model.py /content/LeafGAN/models/leaf_gan_model.py

In [ ]:
!python train.py --dataroot /content/dataset --name GAN4 --model leaf_gan --dataset_mode unaligned_masked --batch_size 4 # 113 103
# !python train.py --dataroot /content/dataset --name GAN7 --model leaf_gan --dataset_mode unaligned_masked
# !python train.py --dataroot /content/dataset --name GAN6 --model leaf_gan --dataset_mode unaligned_masked

----------------- Options ---------------
               batch_size: 4                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /content/dataset              	[default: None]
             dataset_mode: unaligned_masked              	[default: unaligned]
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: -1                            
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest     

In [ ]:
# !python train.py --dataroot /content/dataset --name A2C --model leaf_gan --dataset_mode unaligned_masked  #2022/05/10
# !python train.py --dataroot /content/dataset --name A2B --model leaf_gan --dataset_mode unaligned_masked  #65회 까지만 수행됨

In [ ]:
import time
while True:
    time.sleep(10)

In [ ]:
# content의 학습된 모델을 drive로 옮기는 코드
!mkdir "/content/drive/MyDrive/4-1_Capstone/GAN6"
!cp "/content/LeafGAN/checkpoints/GAN6/latest_net_D_A.pth" "/content/drive/MyDrive/4-1_Capstone/GAN6/latest_net_D_A.pth"
!cp "/content/LeafGAN/checkpoints/GAN6/latest_net_D_B.pth" "/content/drive/MyDrive/4-1_Capstone/GAN6/latest_net_D_B.pth"
!cp "/content/LeafGAN/checkpoints/GAN6/latest_net_G_A.pth" "/content/drive/MyDrive/4-1_Capstone/GAN6/latest_net_G_A.pth"
!cp "/content/LeafGAN/checkpoints/GAN6/latest_net_G_B.pth" "/content/drive/MyDrive/4-1_Capstone/GAN6/latest_net_G_B.pth"

In [ ]:
# content의 이미지를 drive로 옮기는 코드
import shutil
output_filename = "/content/drive/MyDrive/4-1_Capstone/GAN6/result_images"
dir_name = "/content/LeafGAN/checkpoints/GAN6/web/images"
shutil.make_archive(output_filename, 'zip', dir_name)

'/content/drive/MyDrive/4-1_Capstone/GAN6/result_images.zip'